In [1]:
import numpy as np
import statistics
import numpy as np

class NaiveBayes:

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X[y == c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)
            

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = posterior + prior
            posteriors.append(posterior)

        # return class with the highest posterior
        return self._classes[np.argmax(posteriors)]

    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator


In [12]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from ucimlrepo import fetch_ucirepo 
# fetch dataset 
spambase = fetch_ucirepo(id=94) 
  
# data (as pandas dataframes)  
X = spambase.data.features 
y = spambase.data.targets 
X.drop(X.columns[[2,4,5,11,18,20,44,48,49,50,51,52,53]],axis= 1,inplace =True) #Droppin the column containing stopwords and special characters.
X = X.to_numpy()
Y = y.to_numpy()

scaler = StandardScaler()
scaler.fit(X)
standardized_data = scaler.transform(X)

target = spambase.data.targets['Class']

target.shape

features = standardized_data

features.shape

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(features,target,test_size=0.1,random_state=2)

C:\Users\mayan\AppData\Local\Temp\ipykernel_24264\2438174668.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(X.columns[[2,4,5,11,18,20,44,48,49,50,51,52,53]],axis= 1,inplace =True) #Droppin the column containing stopwords and special characters.


In [13]:
np.seterr(divide='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [14]:
nb = NaiveBayes()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

In [15]:
from sklearn import metrics
from sklearn.metrics import classification_report

In [16]:
metrics.confusion_matrix(y_test,y_pred)

array([[202,  69],
       [  8, 182]], dtype=int64)

In [17]:
metrics.f1_score(y_test,y_pred)

0.8253968253968254

In [18]:
metrics.recall_score(y_test,y_pred)

0.9578947368421052

In [19]:
metrics.precision_score(y_test,y_pred)

0.7250996015936255

In [20]:
metrics.accuracy_score(y_test,y_pred)

0.8329718004338394